# Unidad 2.3 - Ruido Sintético y Filtrado Espacial

## Objetivos
- Generar ruido sintético (gaussiano y sal y pimienta)
- Aplicar diferentes filtros espaciales
- Comparar la efectividad de filtros para cada tipo de ruido
- Entender el efecto del tamaño del kernel
- Medir la calidad de la imagen después del filtrado

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Configuración de matplotlib
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

## 1. Cargar imagen de referencia

In [ ]:
# Cargar imagen en escala de grises
img = cv2.imread("imagenes/DPP0357.TIF", cv2.IMREAD_GRAYSCALE)

if img is None:
    print("Error: No se pudo cargar la imagen")
else:
    print(f"Imagen cargada: {img.shape}")
    print(f"Tipo de datos: {img.dtype}")
    print(f"Rango de valores: [{img.min()}, {img.max()}]")
    
    plt.figure(figsize=(8,6))
    plt.imshow(img, cmap='gray')
    plt.title('Imagen Original (Sin ruido)')
    plt.colorbar(label='Intensidad')
    plt.axis('off')
    plt.show()

## 2. Generación de Ruido Gaussiano

El ruido gaussiano añade valores aleatorios con distribución normal a cada píxel.

$$
I_{\text{ruidosa}}(x,y) = I(x,y) + \mathcal{N}(\mu, \sigma^2)
$$

donde $\mathcal{N}(\mu, \sigma^2)$ es una distribución normal con media $\mu$ y desviación estándar $\sigma$.

In [ ]:
def agregar_ruido_gaussiano(imagen, media=0, sigma=25):
    """
    Agrega ruido gaussiano a una imagen.
    
    Parámetros:
        imagen: imagen de entrada (uint8)
        media: media de la distribución gaussiana (típicamente 0)
        sigma: desviación estándar del ruido
    
    Retorna:
        imagen con ruido gaussiano
    """
    # Generar ruido gaussiano del mismo tamaño que la imagen
    ruido = np.random.normal(media, sigma, imagen.shape)
    
    # Añadir ruido a la imagen
    ruidosa = imagen.astype(np.float32) + ruido
    
    # Saturar valores (clip) entre 0 y 255
    ruidosa = np.clip(ruidosa, 0, 255)
    
    # Convertir de vuelta a uint8
    return ruidosa.astype(np.uint8)

# Probar con diferentes niveles de ruido
ruido_bajo = agregar_ruido_gaussiano(img, media=0, sigma=10)
ruido_medio = agregar_ruido_gaussiano(img, media=0, sigma=25)
ruido_alto = agregar_ruido_gaussiano(img, media=0, sigma=50)

print("Ruido gaussiano generado con diferentes niveles de sigma")

In [ ]:
# Visualizar diferentes niveles de ruido gaussiano
plt.figure(figsize=(15, 8))

imagenes_gauss = [
    (img, 'Original\n(sin ruido)'),
    (ruido_bajo, 'Ruido Bajo\n(σ = 10)'),
    (ruido_medio, 'Ruido Medio\n(σ = 25)'),
    (ruido_alto, 'Ruido Alto\n(σ = 50)')
]

for i, (imagen, titulo) in enumerate(imagenes_gauss):
    # Imagen
    plt.subplot(2, 4, i+1)
    plt.imshow(imagen, cmap='gray')
    plt.title(titulo)
    plt.axis('off')
    
    # Histograma
    plt.subplot(2, 4, i+5)
    plt.hist(imagen.ravel(), bins=256, color='steelblue', alpha=0.7)
    plt.title('Histograma')
    plt.xlim([0, 256])
    plt.xlabel('Intensidad')
    plt.ylabel('Frecuencia')

plt.suptitle('Ruido Gaussiano con Diferentes Niveles', fontsize=14)
plt.tight_layout()
plt.show()

## 3. Generación de Ruido Sal y Pimienta

El ruido sal y pimienta reemplaza píxeles aleatorios con valores máximos (255, "sal") o mínimos (0, "pimienta").

In [ ]:
def agregar_ruido_sp(imagen, prob_sal=0.01, prob_pimienta=0.01):
    """
    Agrega ruido sal y pimienta a una imagen.
    
    Parámetros:
        imagen: imagen de entrada (uint8)
        prob_sal: probabilidad de píxeles blancos (sal)
        prob_pimienta: probabilidad de píxeles negros (pimienta)
    
    Retorna:
        imagen con ruido sal y pimienta
    """
    ruidosa = imagen.copy()
    total_pixels = imagen.size
    
    # Sal (píxeles blancos = 255)
    num_sal = int(prob_sal * total_pixels)
    coords_sal = [np.random.randint(0, i, num_sal) for i in imagen.shape]
    ruidosa[coords_sal[0], coords_sal[1]] = 255
    
    # Pimienta (píxeles negros = 0)
    num_pimienta = int(prob_pimienta * total_pixels)
    coords_pimienta = [np.random.randint(0, i, num_pimienta) for i in imagen.shape]
    ruidosa[coords_pimienta[0], coords_pimienta[1]] = 0
    
    return ruidosa

# Probar con diferentes niveles de ruido
np.random.seed(42)  # Para reproducibilidad
sp_bajo = agregar_ruido_sp(img, prob_sal=0.01, prob_pimienta=0.01)
sp_medio = agregar_ruido_sp(img, prob_sal=0.03, prob_pimienta=0.03)
sp_alto = agregar_ruido_sp(img, prob_sal=0.05, prob_pimienta=0.05)

print("Ruido sal y pimienta generado con diferentes niveles")

In [ ]:
# Visualizar diferentes niveles de ruido sal y pimienta
plt.figure(figsize=(15, 8))

imagenes_sp = [
    (img, 'Original\n(sin ruido)'),
    (sp_bajo, 'Ruido Bajo\n(1% cada uno)'),
    (sp_medio, 'Ruido Medio\n(3% cada uno)'),
    (sp_alto, 'Ruido Alto\n(5% cada uno)')
]

for i, (imagen, titulo) in enumerate(imagenes_sp):
    # Imagen
    plt.subplot(2, 4, i+1)
    plt.imshow(imagen, cmap='gray')
    plt.title(titulo)
    plt.axis('off')
    
    # Histograma
    plt.subplot(2, 4, i+5)
    plt.hist(imagen.ravel(), bins=256, color='coral', alpha=0.7)
    plt.title('Histograma')
    plt.xlim([0, 256])
    plt.xlabel('Intensidad')
    plt.ylabel('Frecuencia')

plt.suptitle('Ruido Sal y Pimienta con Diferentes Niveles', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Filtros Espaciales para Eliminación de Ruido

### 4.1 Filtro Promedio (Mean Filter)

In [ ]:
# Aplicar filtro promedio con diferentes tamaños de kernel
# Usaremos ruido_medio como imagen de prueba
kernel_sizes = [3, 5, 7, 9]

plt.figure(figsize=(15, 8))

plt.subplot(2, 5, 1)
plt.imshow(img, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(2, 5, 6)
plt.imshow(ruido_medio, cmap='gray')
plt.title('Con Ruido Gaussiano\n(σ = 25)')
plt.axis('off')

for i, k in enumerate(kernel_sizes):
    filtrada = cv2.blur(ruido_medio, (k, k))
    
    plt.subplot(2, 5, i+2)
    plt.imshow(filtrada, cmap='gray')
    plt.title(f'Filtro Promedio {k}×{k}')
    plt.axis('off')

plt.suptitle('Filtro Promedio sobre Ruido Gaussiano', fontsize=14)
plt.tight_layout()
plt.show()

### 4.2 Filtro Gaussiano

In [ ]:
# Aplicar filtro gaussiano con diferentes tamaños de kernel
plt.figure(figsize=(15, 8))

plt.subplot(2, 5, 1)
plt.imshow(img, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(2, 5, 6)
plt.imshow(ruido_medio, cmap='gray')
plt.title('Con Ruido Gaussiano\n(σ = 25)')
plt.axis('off')

for i, k in enumerate(kernel_sizes):
    # El kernel debe ser impar
    filtrada = cv2.GaussianBlur(ruido_medio, (k, k), 0)
    
    plt.subplot(2, 5, i+2)
    plt.imshow(filtrada, cmap='gray')
    plt.title(f'Filtro Gaussiano {k}×{k}')
    plt.axis('off')

plt.suptitle('Filtro Gaussiano sobre Ruido Gaussiano', fontsize=14)
plt.tight_layout()
plt.show()

### 4.3 Filtro de Mediana

In [ ]:
# Aplicar filtro de mediana con diferentes tamaños de kernel
# El filtro de mediana es particularmente efectivo contra ruido sal y pimienta
plt.figure(figsize=(15, 8))

plt.subplot(2, 5, 1)
plt.imshow(img, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(2, 5, 6)
plt.imshow(sp_medio, cmap='gray')
plt.title('Con Ruido S&P\n(3% cada uno)')
plt.axis('off')

for i, k in enumerate(kernel_sizes):
    filtrada = cv2.medianBlur(sp_medio, k)
    
    plt.subplot(2, 5, i+2)
    plt.imshow(filtrada, cmap='gray')
    plt.title(f'Filtro Mediana {k}×{k}')
    plt.axis('off')

plt.suptitle('Filtro de Mediana sobre Ruido Sal y Pimienta', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Comparación de Filtros: Ruido Gaussiano

In [ ]:
# Comparar los tres filtros sobre ruido gaussiano
k = 5  # Tamaño de kernel fijo para comparación justa

gauss_promedio = cv2.blur(ruido_medio, (k, k))
gauss_gaussiano = cv2.GaussianBlur(ruido_medio, (k, k), 0)
gauss_mediana = cv2.medianBlur(ruido_medio, k)

plt.figure(figsize=(15, 5))

plt.subplot(1, 5, 1)
plt.imshow(img, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 5, 2)
plt.imshow(ruido_medio, cmap='gray')
plt.title('Con Ruido Gaussiano')
plt.axis('off')

plt.subplot(1, 5, 3)
plt.imshow(gauss_promedio, cmap='gray')
plt.title('Filtro Promedio 5×5')
plt.axis('off')

plt.subplot(1, 5, 4)
plt.imshow(gauss_gaussiano, cmap='gray')
plt.title('Filtro Gaussiano 5×5')
plt.axis('off')

plt.subplot(1, 5, 5)
plt.imshow(gauss_mediana, cmap='gray')
plt.title('Filtro Mediana 5×5')
plt.axis('off')

plt.suptitle('Comparación de Filtros sobre Ruido Gaussiano', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Comparación de Filtros: Ruido Sal y Pimienta

In [ ]:
# Comparar los tres filtros sobre ruido sal y pimienta
sp_promedio = cv2.blur(sp_medio, (k, k))
sp_gaussiano = cv2.GaussianBlur(sp_medio, (k, k), 0)
sp_mediana = cv2.medianBlur(sp_medio, k)

plt.figure(figsize=(15, 5))

plt.subplot(1, 5, 1)
plt.imshow(img, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 5, 2)
plt.imshow(sp_medio, cmap='gray')
plt.title('Con Ruido Sal y Pimienta')
plt.axis('off')

plt.subplot(1, 5, 3)
plt.imshow(sp_promedio, cmap='gray')
plt.title('Filtro Promedio 5×5\n(difumina el ruido)')
plt.axis('off')

plt.subplot(1, 5, 4)
plt.imshow(sp_gaussiano, cmap='gray')
plt.title('Filtro Gaussiano 5×5\n(mejora, pero no elimina)')
plt.axis('off')

plt.subplot(1, 5, 5)
plt.imshow(sp_mediana, cmap='gray')
plt.title('Filtro Mediana 5×5\n(elimina casi todo)')
plt.axis('off')

plt.suptitle('Comparación de Filtros sobre Ruido Sal y Pimienta', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Métricas de Calidad: PSNR y MSE

### Mean Squared Error (MSE):
$$
MSE = \frac{1}{MN} \sum_{i=0}^{M-1} \sum_{j=0}^{N-1} [I(i,j) - \hat{I}(i,j)]^2
$$

### Peak Signal-to-Noise Ratio (PSNR):
$$
PSNR = 10 \log_{10} \left( \frac{MAX_I^2}{MSE} \right)
$$

Donde $MAX_I = 255$ para imágenes de 8 bits.

In [ ]:
def calcular_mse(img1, img2):
    """Calcula el Mean Squared Error entre dos imágenes"""
    return np.mean((img1.astype(np.float32) - img2.astype(np.float32)) ** 2)

def calcular_psnr(img1, img2):
    """Calcula el Peak Signal-to-Noise Ratio entre dos imágenes"""
    mse = calcular_mse(img1, img2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

# Calcular métricas para ruido gaussiano
print("=" * 60)
print("MÉTRICAS PARA RUIDO GAUSSIANO (σ=25)")
print("=" * 60)
print(f"Imagen con ruido:")
print(f"  MSE: {calcular_mse(img, ruido_medio):.2f}")
print(f"  PSNR: {calcular_psnr(img, ruido_medio):.2f} dB")
print()
print(f"Después de filtro promedio 5×5:")
print(f"  MSE: {calcular_mse(img, gauss_promedio):.2f}")
print(f"  PSNR: {calcular_psnr(img, gauss_promedio):.2f} dB")
print()
print(f"Después de filtro gaussiano 5×5:")
print(f"  MSE: {calcular_mse(img, gauss_gaussiano):.2f}")
print(f"  PSNR: {calcular_psnr(img, gauss_gaussiano):.2f} dB")
print()
print(f"Después de filtro mediana 5×5:")
print(f"  MSE: {calcular_mse(img, gauss_mediana):.2f}")
print(f"  PSNR: {calcular_psnr(img, gauss_mediana):.2f} dB")
print()
print("=" * 60)
print("MÉTRICAS PARA RUIDO SAL Y PIMIENTA (3%)")
print("=" * 60)
print(f"Imagen con ruido:")
print(f"  MSE: {calcular_mse(img, sp_medio):.2f}")
print(f"  PSNR: {calcular_psnr(img, sp_medio):.2f} dB")
print()
print(f"Después de filtro promedio 5×5:")
print(f"  MSE: {calcular_mse(img, sp_promedio):.2f}")
print(f"  PSNR: {calcular_psnr(img, sp_promedio):.2f} dB")
print()
print(f"Después de filtro gaussiano 5×5:")
print(f"  MSE: {calcular_mse(img, sp_gaussiano):.2f}")
print(f"  PSNR: {calcular_psnr(img, sp_gaussiano):.2f} dB")
print()
print(f"Después de filtro mediana 5×5:")
print(f"  MSE: {calcular_mse(img, sp_mediana):.2f}")
print(f"  PSNR: {calcular_psnr(img, sp_mediana):.2f} dB")
print("=" * 60)

## 8. Resumen Visual de Resultados

In [ ]:
# Crear un resumen visual completo
fig, axes = plt.subplots(3, 5, figsize=(18, 11))

# Fila 1: Original y ruido gaussiano
axes[0,0].imshow(img, cmap='gray')
axes[0,0].set_title('Original', fontsize=12)
axes[0,0].axis('off')

axes[0,1].imshow(ruido_medio, cmap='gray')
axes[0,1].set_title(f'Ruido Gaussiano\nPSNR: {calcular_psnr(img, ruido_medio):.1f} dB', fontsize=11)
axes[0,1].axis('off')

axes[0,2].imshow(gauss_promedio, cmap='gray')
axes[0,2].set_title(f'Promedio\nPSNR: {calcular_psnr(img, gauss_promedio):.1f} dB', fontsize=11)
axes[0,2].axis('off')

axes[0,3].imshow(gauss_gaussiano, cmap='gray')
axes[0,3].set_title(f'Gaussiano\nPSNR: {calcular_psnr(img, gauss_gaussiano):.1f} dB', fontsize=11)
axes[0,3].axis('off')

axes[0,4].imshow(gauss_mediana, cmap='gray')
axes[0,4].set_title(f'Mediana\nPSNR: {calcular_psnr(img, gauss_mediana):.1f} dB', fontsize=11)
axes[0,4].axis('off')

# Fila 2: Original y ruido sal y pimienta
axes[1,0].imshow(img, cmap='gray')
axes[1,0].set_title('Original', fontsize=12)
axes[1,0].axis('off')

axes[1,1].imshow(sp_medio, cmap='gray')
axes[1,1].set_title(f'Ruido Sal & Pimienta\nPSNR: {calcular_psnr(img, sp_medio):.1f} dB', fontsize=11)
axes[1,1].axis('off')

axes[1,2].imshow(sp_promedio, cmap='gray')
axes[1,2].set_title(f'Promedio\nPSNR: {calcular_psnr(img, sp_promedio):.1f} dB', fontsize=11)
axes[1,2].axis('off')

axes[1,3].imshow(sp_gaussiano, cmap='gray')
axes[1,3].set_title(f'Gaussiano\nPSNR: {calcular_psnr(img, sp_gaussiano):.1f} dB', fontsize=11)
axes[1,3].axis('off')

axes[1,4].imshow(sp_mediana, cmap='gray')
axes[1,4].set_title(f'Mediana\nPSNR: {calcular_psnr(img, sp_mediana):.1f} dB', fontsize=11)
axes[1,4].axis('off')

# Fila 3: Diferencias (errores)
axes[2,0].axis('off')

axes[2,1].imshow(np.abs(img.astype(int) - ruido_medio.astype(int)), cmap='hot')
axes[2,1].set_title('Error: Con ruido', fontsize=11)
axes[2,1].axis('off')

axes[2,2].imshow(np.abs(img.astype(int) - gauss_promedio.astype(int)), cmap='hot')
axes[2,2].set_title('Error: Promedio', fontsize=11)
axes[2,2].axis('off')

axes[2,3].imshow(np.abs(img.astype(int) - gauss_gaussiano.astype(int)), cmap='hot')
axes[2,3].set_title('Error: Gaussiano', fontsize=11)
axes[2,3].axis('off')

axes[2,4].imshow(np.abs(img.astype(int) - gauss_mediana.astype(int)), cmap='hot')
axes[2,4].set_title('Error: Mediana', fontsize=11)
axes[2,4].axis('off')

plt.suptitle('Comparación Completa: Tipos de Ruido y Filtros (kernel 5×5)', fontsize=14)
plt.tight_layout()
plt.show()

## 9. Ejercicio Práctico

Carga tu propia imagen y experimenta con diferentes tipos y niveles de ruido:

In [ ]:
# TODO: Cargar tu imagen
# mi_imagen = cv2.imread("ruta/a/tu/imagen.jpg", cv2.IMREAD_GRAYSCALE)

# TODO: Agregar ruido
# 1. Probar diferentes niveles de ruido gaussiano
# 2. Probar diferentes niveles de ruido sal y pimienta
# 3. Combinar ambos tipos de ruido

# TODO: Aplicar filtros
# 1. Probar diferentes filtros
# 2. Probar diferentes tamaños de kernel
# 3. Calcular métricas PSNR

# TODO: Análisis
# 1. ¿Qué filtro funciona mejor para cada tipo de ruido?
# 2. ¿Cómo afecta el tamaño del kernel al resultado?
# 3. ¿Hay algún trade-off entre eliminación de ruido y preservación de detalles?

print("Completa este ejercicio con tu propia imagen")

## Conclusiones

### Tipos de Ruido:
- **Ruido Gaussiano**: Añade valores aleatorios con distribución normal (común en sensores)
- **Ruido Sal y Pimienta**: Reemplaza píxeles con valores extremos (común en transmisión)

### Efectividad de Filtros:
- **Para Ruido Gaussiano**:
  - Filtro Gaussiano es el más efectivo (suavizado natural)
  - Filtro Promedio también funciona bien
  - Filtro de Mediana preserva mejor los bordes pero puede ser insuficiente

- **Para Ruido Sal y Pimienta**:
  - **Filtro de Mediana es el mejor** (elimina casi completamente el ruido)
  - Filtros Promedio y Gaussiano solo difuminan el ruido

### Trade-offs:
- Kernels más grandes eliminan más ruido pero difuminan más la imagen
- Es importante equilibrar eliminación de ruido con preservación de detalles
- PSNR más alto indica mejor calidad de reconstrucción